---
title: "Machine Learning in Python"
subtitle: "Applied Machine Learning"
author: "Calvin"
  
jupyter: python3

execute:
    echo: true
    cache: true
    freeze: true  # never re-render during project render
---


## Prerequisite

- I assume a Python/VS Code/Quarto workflow.
    - Review [this document](Python00.html)
- I assume familiarity with Machine Learning in R
    - Review [these slides](Session01.html)


## Agenda

1.  Python Overview
2.  Review of Regression
3.  Classification
4.  Basic Feature Engineering

## Quarto

-   I switch from a R backend to a Python backend.
-   I add the following below my title in my .qmd header:
    -   `jupyter: python3`

```yml         
title: "Machine Learning in Python"
subtitle: "Applied Machine Learning"
author: "Calvin"
  
jupyter: python3
```

## Pip

-   In Python, we can typically install packages via `pip`
-   It is more typical to use `pip` at commandline.

``` bash
python -m pip install sampleproject
```

-   Here is a 'clean' way to do so from within the Python


In [ ]:
import subprocess  # A base package we need to install other packages
import sys         # A base package we need to install other packages
install = lambda package : subprocess.check_call([sys.executable, 
                                                  '-m', 
                                                  'pip', 
                                                  'install', 
                                                  package])

## Packages

-   I'll build a list of packages then install them via a loop.
    -   Some (numpy, matplotlib) required for Quarto.


In [ ]:
python_data_stack = ["numpy", "pandas", "matplotlib", "seaborn"]
r_data_stack = ['pyreadr', 'statsmodels']
packages = python_data_stack + r_data_stack + ["scikit-learn"]

_ = [install(package) for package in packages]

-   I use `_ =` to discard the result of the process.
    -   This ignores errors - remove to debug.

## Import

-   Python packages use `import` rather than `library()`
-   Python base data stack

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

-   R & statistics

In [ ]:
import pyreadr
import statsmodels.api as sm
import statsmodels.formula.api as smf

## ML Library
-   There are 3.5 Python ML libraries that matter
    -   Scikit-learn, mainline ML
    -   Torch, deep learning
    -   Tensorflow, deep learning
    -   PySpark MLlib, MLOps

In [ ]:
# Common to include parts of, not all of, sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Many packages
-   There's a lot of imports here, we could cut back but...
-   We would typical use only `sklearn` not `stats_models`
    -   SM=statisical learning, SK=machine learning
    -   Use SM for equivalence with R.
-   Typically use the module name as a function's prefix.
    -   This resolves name collisions, like `dplyr` and `stats`

    ``` r
    ✖ dplyr::filter() masks stats::filter()
    ✖ dplyr::lag()    masks stats::lag()
    ```

## Setup

-   We will use `pyreadr` to read in an R dataset.
    -   This is atypical but not difficult.


In [ ]:
url = 'https://cd-public.github.io/D505/dat/'
rds = 'wine.rds'
pyreadr.download_file(url + rds, rds) 
wine = pyreadr.read_r(rds)[None]      
wine.dropna(subset=['points','price'])
str(wine.info()) # string for slide formatting

# Review of Regression

## Single Variable

-   Pick the poshest province.


In [ ]:
wine['bordeaux'] = wine['province'] == 'Bordeaux'
wine.head(2)

## Regress

-   Take a quick regression model over the wine.


In [ ]:
m1 = sm.OLS(wine['points'], wine['price']).fit()
m1.summary()

## Sklearn

-   I'm much fonder of `sklearn` for this sort of thing.


In [ ]:
s1 = LinearRegression().fit(wine[['points']],wine['price'])
f'Intercept: {s1.intercept_}, Coefficients: {s1.coef_}'

## Let's draw it

:::: {.columns}

::: {.column width='50%'}
-   Python ggplot is `plotnine`
-   I prefer Seaborn (`sns`)
-   Also: `matplotlib`
-   Also: `plotly`
:::

::: {.column width='50%'}

In [ ]:
sns.lmplot(x='points', y='price', 
           data=wine, scatter=True, 
           x_bins=range(80,101))

:::

::::

## Multiple regression

-   We can literally use the exact same formula in `smf`
-   I am unaccustomed to doing multiple regressions without interaction.
    -   Moving on.


In [ ]:
m2 = smf.ols('price ~ points + bordeaux', data=wine).fit()
m2.summary()

## How about with an interaction?


In [ ]:
m3 = smf.ols('price ~ points * bordeaux', data=wine).fit()
m3.summary()

## Sklearinteraction

-   `sklearn` defaults to multiple regression.


In [ ]:
wine['points_bordeaux'] = wine['points'] * wine['bordeaux']

s3 = LinearRegression().fit(wine[['points','bordeaux']],
                            wine['price'])
f'Intercept: {s3.intercept_}, Coefficients: {s3.coef_}'

## Let's draw it


In [ ]:
sns.lmplot(x='points', y='price', 
           data=wine, scatter=True, 
           x_bins=range(80,101), hue='bordeaux')

# Moving to an ML framework

## Split sample using Sklearn
-   Vs R, with a proliferation of libraries, Python ML is concentrated.
-   There are a few main libraries, `Sklearn` the first and most popular
-   So modelling and partitioning are all in `sklearn`, vs `stats` and `caret`


In [ ]:
random_seed = 505
train, test = train_test_split(wine, random_state=random_seed)
print(test.head())

## Compare RMSE across models

-   Retrain the models on the training set only


In [ ]:
formulas = ['price ~ points', 
            'price ~ points + bordeaux', 
            'price ~ points * bordeaux']
ms = [smf.ols(f, data=train).fit() for f in formulas]

-   Test them all under the same conditions.


In [ ]:
def rmse(m):
  residuals_sq = (test['price'] - m.predict(test)) ** 2
  mse = np.mean(residuals_sq)
  return mse ** .5

[rmse(m) for m in ms]

## Exercise Problems

-   *Bonus++. Pick a non-Bordeaux category.*
    -   We write a quick reserve-checker and apply it.
    -   Basically Python wasn't told all the data was textual
        -   Use `str` and `lower`
    -   Then use Pythonic `in` with a list of names
-   `apply` is like `mutate`, in a way.


In [ ]:
is_reserve = lambda x: str(x).lower() in ['reserve','reserva','riserva']
wine['reserve'] = wine['designation'].apply(is_reserve)

## via Seaborn


In [ ]:
sns.lmplot(wine, x='points', y='price', x_bins=range(80,101), hue='reserve',
           x_estimator=np.mean, order=2) # polynomail regression, why not

# Classification and Ethics

## The math of it...

-   This problem used a R library to get data.
-   That R library is open source, 
    -   It has reasonable documentation, and
    -   It's GitHub is linked from documentation.
-   I spent 30 seconds on a search engine to find it.

## Partition our Data
-   The `.rda` file is at this url:
    -   [https://github.com/rafalab/dslabs/blob/master/data/heights.rda](https://github.com/rafalab/dslabs/blob/master/data/heights.rda){style="font-size:smaller"}
    -   Change `blob` to `raw` to [download directly](https://github.com/rafalab/dslabs/raw/master/data/heights.rda).


In [ ]:
url = 'https://github.com/rafalab/dslabs/raw/master/data/'
rds = 'heights.rda'
pyreadr.download_file(url + rds, rds) 
heights = pyreadr.read_r(rds)['heights']      
random_seed = 505
train, test = train_test_split(heights, random_state=random_seed)

Note: this vignette is adapted from [this book](https://rafalab.github.io/dsbook/introduction-to-machine-learning.html)

## Guessing
-   Let’s start by developing the simplest possible machine algorithm: guessing the outcome.

In [ ]:
y_hat = np.random.choice(['Male', 'Female'], len(test))

Recall:

>[Y hat (written ŷ ) is the predicted value of y (the dependent variable) in a regression equation. It can also be considered to be the average value of the response variable.](https://www.statisticshowto.com/y-hat-definition/)

## Accuracy
-   The overall accuracy is simply defined as the overall proportion that is predicted correctly:

In [ ]:
np.mean(y_hat == test['sex'])

-   What would we have expected the accuracy to be?
    -   What much would we have expected accuracy to deviate from that expectionation?

## Let's do better...


In [ ]:
hs = heights.groupby('sex')['height']
hs.std(), hs.mean()

## A simple predictive model

-   Idea: Predict `'Male'` if observation is within 2 standard deviations


In [ ]:
male_mean_less_2sd = hs.mean()['Male'] - 2*hs.std()['Male']

y_hat = heights['height'].apply(lambda x : 'Male' if x > male_mean_less_2sd else 'Female')

print(male_mean_less_2sd, np.mean(heights['sex'] == y_hat))

-   The accuracy goes up from \~0.50 to about \~0.80!!

## Let's optimize


In [ ]:
cutoff = list(range(61,71))

def get_accuracy(y):
  f = lambda x : 'Male' if x > y else 'Female'
  y_hat = heights['height'].apply(f)
  return np.mean(heights['sex'] == y_hat)

accuracy = list(map(get_accuracy, cutoff))

accuracy

## Optimal Cutoff


In [ ]:
best_cutoff = cutoff[np.argmax(accuracy)]
_ = plt.plot(cutoff, accuracy), print('Optimal cutoff is', best_cutoff)

-   Should we be cutting at an integer?

## Apply & Evaluate


In [ ]:
test['y_hat'] = test['height'].apply(lambda x : 'Male' if x > best_cutoff else 'Female')
print('Accuracy is', np.mean(test['sex'] == test['y_hat']))

## Confusion matrix

In [ ]:
ConfusionMatrixDisplay.from_predictions(test['sex'], test['y_hat'])

*Took 7 lines / 350+ characters of handwritten ggplot*

## Accuracy by sex


In [ ]:
test['acc'] = test['sex'] == test['y_hat']
test.groupby('sex')['acc'].mean()

It's raining men.

## Debrief


:::: {.columns}

::: {.column width="50%"}

In [ ]:
_ = sns.boxplot(heights, x='sex',y='height')

:::

::: {.column width="50%"}

In [ ]:
_ = plt.pie(heights['sex'].value_counts(), labels=['♀','♂'])

:::

::::

# Machine Learning in Python